## 뉴스 크롤링 코드
- 검색어 : ex. 뉴진스
- 검색 시작일 : ex. 20230601
- 검색 종료일 : ex. 20230630
- 시작 페이지 : ex.1
- 종료 페이지 : ex.200


In [ ]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&pd=3&ds=" + start_date + "&de=" + end_date + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&pd=3&ds=" + start_date + "&de=" + end_date + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색시작 날짜 입력
start_date = input("검색시작 날짜를 입력해주세요:")
#검색종료 날짜 입력
end_date = input("검색종료 날짜를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    try:
        url = articles_crawler(url)
        news_url.append(url)
    except:
        print('기사를 건너뜁니다')




#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist



#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

# #모든 기사 담기
# final_urls = []
# for i in tqdm(range(len(news_url_1))):
#     final_urls.append(news_url_1[i])


# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # if search in title.get_text(): 타이들이 있는 경우


    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

## 블로그 크롤링 코드

In [ ]:
# 코랩에서 돌릴 경우 필요한 코드

# %%shell
# sudo apt -y update
# sudo apt install -y wget curl unzip
# wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
# dpkg -i libu2f-udev_1.1.4-1_all.deb
# wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# dpkg -i google-chrome-stable_current_amd64.deb
# CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
# wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
# unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
# chmod +x /tmp/chromedriver
# mv /tmp/chromedriver /usr/local/bin/chromedriver

# selenium, bs4 설치

# pip install selenium
# !pip install bs4

#크롤링시 필요한 라이브러리 불러오기

import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
from selenium.webdriver.common.by import By
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

# 검색어
keyword = input("크롤링할 키워드를 입력하세요: ")

# 크롬 실행
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)
driver.get("http://www.naver.com")
time.sleep(2)

# 검색창에 '검색어' 검색
element = driver.find_element(By.ID, 'query') #f12눌러서 확인
element.send_keys(keyword)
element.submit()
time.sleep(1)
# 'VIEW' 클릭
driver.find_element(By.PARTIAL_LINK_TEXT, "VIEW").click()
# '블로그' 클릭
driver.find_element(By.PARTIAL_LINK_TEXT, "블로그").click()
time.sleep(1)
# '옵션' 클릭
driver.find_element(By.PARTIAL_LINK_TEXT, "옵션").click()
time.sleep(0.5)
# 검색옵션 확인
item_li = driver.find_elements(By.CSS_SELECTOR,".option .txt")

for i in range(0, len(item_li)):
    print(item_li[i].text)

# 검색기간 '3개월' 클릭
print(item_li[10].text)

item_li[10].click()

# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 5
i = 0
while i < n:
    scroll_down(driver)
    i = i+1

# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
article_raw = driver.find_elements(By.CSS_SELECTOR,".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')
    url_list.append(url)

time.sleep(1)

# 제목 크롤링 시작
for article in article_raw:
    title = article.text
    title_list.append(title)

print("url개수: ", len(url_list))
print("title개수: ", len(title_list))

url_list

title_list

df = pd.DataFrame({'url':url_list, 'title':title_list})
df

# url_list 저장
df.to_csv("naver_blog_url.csv")

### Step 2. 블로그 내용 크롤링

# "url_list.csv" 불러오기
url_load = pd.read_csv("naver_blog_url.csv")

num_list = len(url_load)

print(num_list)
url_load

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = num_list    # 수집할 글 갯수

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load["url"][i]
    print(url)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)
    driver.get(url)   # 글 띄우기

    # 크롤링

    try :
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"
        tit = driver.find_element(By.CSS_SELECTOR,overlays)
        title = tit.text
        # print(title)
        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"
        date = driver.find_element(By.CSS_SELECTOR,overlays)
        datetime = date.text
        # print(datetime)
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"
        contents = driver.find_elements(By.CSS_SELECTOR,overlays)
        # print(contents)
        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)

        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['title'] = title
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)

        # 크롤링 성공하면 글 제목을 출력
        # print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()

    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue

    import pandas as pd
    result_df = pd.DataFrame.from_dict(dict, 'index')
    # 저장하기
    result_df.to_csv("naver_blog_content.csv", encoding='utf-8-sig')   # 한글은 오류날 수 있어서 encoding 추가
    time.sleep(3)

print('수집한 글 갯수: ', len(dict))

dict[0]

# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

# 엑셀 저장
result_df.to_csv("naver_blog_content.csv", encoding='utf-8-sig')

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602629 sha256=fadf16f02100f79f5c21ac65f15bfea1e35fdcc61cc1a59c3032d40a23eb9604
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.7 MB/s eta 0:00:00


# 광고 모델 키워드 추출 데이터셋
- 뉴진스 데이터를 통해 이미지와 관련된 키워드 추출
- 이를 블랙핑크 데이터에도 적용
```
뉴진스 : train
블랙핑크 : test
```

## 1) 웹 크롤링으로 비정형 데이터 수집
- 23년 5~7월 총 3개월 간의 뉴진스와 블랙핑크의 뉴스, 블로그 자료 수집
- 텍스트 정제

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re

DATA_PATH = "/content/drive/MyDrive/project_semi/data/"
SEED = 42

#### 뉴진스 데이터

In [ ]:
train_1 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_05.csv")
train_2 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_06.csv")
train_3 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_07.csv")
train_4 = pd.read_csv(f"{DATA_PATH}뉴진스_블로그_3개월.csv", encoding='cp949')

In [ ]:
train_1.shape , train_2.shape , train_3.shape , train_4.shape

((1030, 4), (912, 4), (595, 4), (148, 4))

In [ ]:
train = pd.concat([train_1,train_2,train_3,train_4],axis=0)
train.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오전 10:06,\n\t\t\t'그래미 5관왕' 존 바티스트 신곡에 뉴진스 협업\n\t\t,https://n.news.naver.com/mnews/article/003/001...,[\n'코크 스튜디오' 위해 만든 싱글 '비 후 유 아' 작업 \n[서울=뉴시스] ...,NaN,NaN
1,2023.05.31. 오후 3:31,"\n\t\t\t""'품위녀'와 전혀 달라"" K-맘 뉴진스의 '행복배틀'[종합]\n\t\t",https://n.news.naver.com/mnews/article/437/000...,[\n \n31일 오후 서울 마포구 상암동 스탠포드호텔에서 ENA '행복배틀'...,NaN,NaN
2,2023.05.31. 오후 3:40,"\n\t\t\t뉴진스, 블랙핑크 그리고 ‘행복배틀’…이엘·진서연→우정원, 케이맘이 ...",https://n.news.naver.com/mnews/article/057/000...,"[\n \n‘행복배틀’ 제작발표회 사진=김영구 MK스포츠 기자‘행복배틀’ 이엘, 진...",NaN,NaN
3,2023.05.31. 오후 3:16,"\n\t\t\t""뉴진스·블랙핑크 이전에 K-맘"" 진서연→이엘 '행복배틀' 발발[종합...",https://n.news.naver.com/mnews/article/108/000...,[\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t[스타뉴스 | 상암(서울)=...,NaN,NaN
4,2023.05.31. 오후 6:25,\n\t\t\t뉴진스·아이브·에스파 이어 (여자)아이들…'퀸카' 멜론 일간차트 1위...,https://n.news.naver.com/mnews/article/003/001...,"[\n미니 6집 '아이 필', '빌보드 200' 41위 자체 최고 기록 \n[서울=...",NaN,NaN


In [ ]:
train["title"].isnull().sum() , train["content"].isnull().sum()

(0, 0)

In [ ]:
train_title = []
for title in train['title']:
    pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
    result = pat.sub(" ",title)
    result = re.sub(" +", " ", result)
    train_title.append(result)

train_title

[' 그래미 5관왕 존 바티스트 신곡에 뉴진스 협업 ',
 ' 품위녀 와 전혀 달라 K 맘 뉴진스의 행복배틀 종합 ',
 ' 뉴진스 블랙핑크 그리고 행복배틀 이엘 진서연 우정원 케이맘이 온다 종합 M 이슈 ',
 ' 뉴진스 블랙핑크 이전에 K 맘 진서연 이엘 행복배틀 발발 종합 ',
 ' 뉴진스 아이브 에스파 이어 여자 아이들 퀸카 멜론 일간차트 1위 ',
 ' 논란의 인어공주 뉴진스 다니엘 비하인드 공개 ',
 ' 블랙핑크 뉴진스 전에 K맘 행복배틀 이엘 진서연의 스릴러 종합 ',
 ' 신현준 뉴진스 하입보이 커버댄스 황제성 형은 하야시 보이 ',
 ' 행복배틀 진서연 뉴진스 블랙핑크 전에 K 맘 있었다 MK 현장 ',
 ' 목표 신인상 보이넥스트도어 뉴진스 이어 트리플 타이틀로 공략 ',
 ' 올해 멜론 주간차트 1위는 세팀 뉴진스 아이브 이어 에스파 ',
 '뉴진스 타임 차세대 리더 로 피프티 피프티 핫 100 9주 머물러',
 ' 뉴진스 발표 7곡 스포티파이 합산 누적 스트리밍 15억 돌파 ',
 ' 뉴진스 미 패션지 표지 장식 전례 없는 K팝 그룹 ',
 ' 뉴진스 다니엘 인어공주 테스트 통과한 이유 있었네 엑 s 리뷰 ',
 ' 뉴진스 첫 팬미팅 초고속 매진 ',
 ' 아이브 뉴진스 춤추는 윤아라니 이게 소녀시대 센터다 임윤아 오피셜 ',
 ' 엔터테인 amp 밈 또 다른 K팝 예술 ',
 ' 윤아 역시 소녀시대 센터 아이브 뉴진스 춤 클라스 눈으로 익혀 임윤아 오피셜 ',
 ' 반전의 인어공주 뉴진스 다니엘 더빙판 호평 첫주 2500억원 흥행수익 돌파 SS무비 ',
 ' 임윤아가 아이브 뉴진스 춤을 추면 소녀시대 센터의 위엄 ',
 ' 임윤아 소녀시대 센터의 자질 뉴진스로 갓데뷔한 줄 ',
 '맥도날드 아시아 10개국서 뉴진스 치킨 댄스 캠페인 공개',
 ' 뉴진스 타임 2023 차세대 리더 선정 ',
 ' D 현장 역시 라는 말 듣고 싶어 보이넥스트도어가 짊어진 책임감 ',
 ' 뉴진스 커버까지 김우석 서울 팬콘 성료 투어 스타트 ',
 ' 뉴진

In [ ]:
train_content = []
for content in train['content']:
    pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
    result = pat.sub(" ",content)
    result = re.sub(" +", " ", result)
    train_content.append(result)

train_content

[' 코크 스튜디오 위해 만든 싱글 비 후 유 아 작업 서울 뉴시스 존 바티스트 2023 05 31 사진 유니버설 뮤직 제공 photo newsis com 재판매 및 DB 금지 서울 뉴시스 이재훈 기자 제64회 그래미 어워즈 최다 수상자인 미국 재즈 뮤지션 존 바티스트 Jon Batiste 의 신곡에 신드롬 걸그룹 뉴진스 NewJeans 가 참여했다 31일 유니버설뮤직에 따르면 바티스트가 코카콜라의 코크 스튜디오 를 위해 만든 싱글 비 후 유 아 Be Who You Are Real Magic 에 뉴진스가 피처링으로 참여했다 뉴진스는 앞서 코크 스튜디오와 협업곡 제로 Zero 를 발표하기도 했다 바티스트의 비 후 유 아 엔 뉴진스 외에 라틴 팝 아티스트 카밀로 Camilo 싱어송라이터 캣 번스 Cat Burns 래퍼 제이아이디 J I D 등도 협업한다 재즈 피아니스트 겨 보컬인 바티스트는 2021년 정규 앨범 위 아 We Are 로 작년 4월 제64회 그래미 어워즈 에서 올해의 앨범상을 포함해 5개 부문을 수상하면서 최다 수상자 로 등극했다 전통 재즈 기반 위에 가스펠 솔 힙합 등 다양한 장르를 폭넓게 녹여낸 생동감 넘치는 작품 이라는 호평을 들었다 서울 뉴시스 뉴진스 2023 04 03 사진 어도어 제공 photo newsis com 재판매 및 DB 금지또 바티스트는 전설 허비 행콕 Herbie Hancock 에 이어 역사상 두 번째로 아카데미 작곡상을 받은 흑인 작곡가이기도 하다 그는 2020년 디즈니 픽사 영화 소울 오리지널 스코어 참여로 아카데미 음악상을 받는 영예를 안았다 영국 아카데미 시상식 골든글로브 시상식 크티틱스 초이스 시상식 그리고 NAACP 이미지 시상식에서도 수상했다 지난 5일엔 명성 있는 뉴올리언스 재즈 페스티벌 헤드라이너로서 무대를 빛냈다 오는 8월에는 뉴포트 재즈 페스티벌의 헤드라인을 장식한다 또한 미국 NBC 유명 모닝쇼 투데이 쇼 의 서머 콘서트 시리즈에도 참가할 예정이다 재즈에 관심 있는 글로벌 슈퍼 그룹 방탄소년단 BTS 멤버 

In [ ]:
train['title'] = train_title
train['content'] = train_content
train.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오전 10:06,그래미 5관왕 존 바티스트 신곡에 뉴진스 협업,https://n.news.naver.com/mnews/article/003/001...,코크 스튜디오 위해 만든 싱글 비 후 유 아 작업 서울 뉴시스 존 바티스트 202...,NaN,NaN
1,2023.05.31. 오후 3:31,품위녀 와 전혀 달라 K 맘 뉴진스의 행복배틀 종합,https://n.news.naver.com/mnews/article/437/000...,31일 오후 서울 마포구 상암동 스탠포드호텔에서 ENA 행복배틀 제작발표회가 열렸...,NaN,NaN
2,2023.05.31. 오후 3:40,뉴진스 블랙핑크 그리고 행복배틀 이엘 진서연 우정원 케이맘이 온다 종합 M 이슈,https://n.news.naver.com/mnews/article/057/000...,행복배틀 제작발표회 사진 김영구 MK스포츠 기자 행복배틀 이엘 진서연 차예련 박효...,NaN,NaN
3,2023.05.31. 오후 3:16,뉴진스 블랙핑크 이전에 K 맘 진서연 이엘 행복배틀 발발 종합,https://n.news.naver.com/mnews/article/108/000...,스타뉴스 상암 서울 김노을 기자 사진제공 ENA타인에게 과시하고 싶은 욕망 아이들...,NaN,NaN
4,2023.05.31. 오후 6:25,뉴진스 아이브 에스파 이어 여자 아이들 퀸카 멜론 일간차트 1위,https://n.news.naver.com/mnews/article/003/001...,미니 6집 아이 필 빌보드 200 41위 자체 최고 기록 서울 뉴시스 여자 아이들...,NaN,NaN


In [ ]:
# 제목, 본문내용 합치기
train_text = train['title'] + train['content']
train_text

0       그래미 5관왕 존 바티스트 신곡에 뉴진스 협업  코크 스튜디오 위해 만든 싱글 비...
1       품위녀 와 전혀 달라 K 맘 뉴진스의 행복배틀 종합  31일 오후 서울 마포구 상...
2       뉴진스 블랙핑크 그리고 행복배틀 이엘 진서연 우정원 케이맘이 온다 종합 M 이슈 ...
3       뉴진스 블랙핑크 이전에 K 맘 진서연 이엘 행복배틀 발발 종합  스타뉴스 상암 서...
4       뉴진스 아이브 에스파 이어 여자 아이들 퀸카 멜론 일간차트 1위  미니 6집 아이...
                             ...                        
143    뉴진스 토끼티 포토카드 앨범기밀이라고 해서 거참 어디가서 말할 수는 없지만 뉴진스 ...
144    오렌즈 프렌츠샤인 원데이 웜톤 쿨톤을 위한 뉴진스 렌즈오렌즈에서 프렌츠샤인이라는 신...
145    뉴진스 파워퍼프걸 캐릭터 만들기 테스트 링크 mbti 의외로 ENFJ미국의 키치한 ...
146    뉴진스의 사인 축구 셔츠를 받을 수 있는 이벤트 공개 with 카포스토어대한민국의 ...
147    뉴진스 파워퍼프걸 콜라보 케이스티파이 powerpuff yourself 파워퍼프걸 ...
Length: 2685, dtype: object

### 블랙핑크 데이터

In [ ]:
test_1 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_5월.csv")
test_2 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_6월.csv")
test_3 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_7월.csv")
test_4 = pd.read_csv(f"{DATA_PATH}블랙핑크_블로그_3개월.csv", encoding='utf-8')

In [ ]:
test_1.shape , test_2.shape , test_3.shape , test_4.shape

((1125, 4), (387, 4), (235, 4), (180, 4))

In [ ]:
test = pd.concat([test_1,test_2,test_3,test_4],axis=0)
test.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오후 9:17,\n\t\t\t블랙핑크와 게임의 만남\n\t\t,https://n.news.naver.com/mnews/article/469/000...,"[\n'블랙핑크 더 게임', 지난 18일 글로벌 출시""블랙핑크 매력 보여줄 수 있는...",NaN,NaN
1,2023-05-31 20:00:01,[쉬운 우리말 쓰기] 블랙핑크가 맡은 '헤드라이너'가 뭔가요?,https://n.news.naver.com/mnews/article/001/001...,[\n\t\t\t[쉬운 우리말 쓰기] 블랙핑크가 맡은 '헤드라이너'가 뭔가요?[ ※...,NaN,NaN
2,2023-05-30 08:29:03,"유안타證 ""와이지엔터, 블랙핑크 성장세 지속""",https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n(자료제공=유안타증권) *재판매 및 DB 금지[서울=뉴시스]이지...,NaN,NaN
3,2023.05.30. 오전 9:05,"\n\t\t\t'블랙핑크 여동생' 베이비 몬스터, 심상치 않은 인기..글로벌팬 공감...",https://n.news.naver.com/mnews/article/112/000...,[\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t[헤럴드POP=김지혜 기자]...,NaN,NaN
4,2023.05.31. 오후 3:11,"\n\t\t\t""블랙핑크·뉴진스 전에 K맘""…'행복배틀' 이엘→진서연의 스릴러 [종...",https://n.news.naver.com/mnews/article/311/000...,[\n \n(엑스포츠뉴스 김현정 기자) ‘행복배틀’이 치열한 심리전과 예측불가 전개...,NaN,NaN


In [ ]:
test["title"].isnull().sum() , test["content"].isnull().sum()

(0, 3)

In [ ]:
test["content"] = test["content"].fillna("UNK")
test["content"].isnull().sum()

0

In [ ]:
test_title = []
for title in test['title']:
    pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
    result = pat.sub(" ",title)
    result = re.sub(" +", " ", result)
    test_title.append(result)

test_title

[' 블랙핑크와 게임의 만남 ',
 ' 쉬운 우리말 쓰기 블랙핑크가 맡은 헤드라이너 가 뭔가요 ',
 '유안타 와이지엔터 블랙핑크 성장세 지속 ',
 ' 블랙핑크 여동생 베이비 몬스터 심상치 않은 인기 글로벌팬 공감대 ',
 ' 블랙핑크 뉴진스 전에 K맘 행복배틀 이엘 진서연의 스릴러 종합 ',
 ' 뉴진스 블랙핑크 그리고 행복배틀 이엘 진서연 우정원 케이맘이 온다 종합 M 이슈 ',
 ' 블랙핑크 제니 제니표 거울 셀카 ',
 ' 뉴진스 블랙핑크 이전에 K 맘 진서연 이엘 행복배틀 발발 종합 ',
 ' 방탄소년단 1위 5월 스타 브랜드평판 TOP 2위 임영웅 3위 블랙핑크 ',
 ' 블랙핑크 제니 무대 밖 나 평범하고 수줍은 소녀일 뿐 스타이슈 ',
 ' 제니 배우 아닌 블랙핑크로 존재 자체가 HIP 똑똑SNS ',
 ' 행복배틀 진서연 뉴진스 블랙핑크 전에 K 맘 있었다 MK 현장 ',
 ' 피프티 피프티 큐피드 트윈버전 스포티파이 3억 스트리밍 돌파 블랙핑크 이어 K팝 걸그룹 두 번째 최단기록 ',
 ' 제니 블랙핑크 디 아이돌 촬영 응원 극 중 춤 장면 많을 것 귀띔 ',
 ' 블랙핑크 지수 활짝 피어나 스타 샷 ',
 ' 블랙핑크 로제x고소영 25살 뛰어넘었다 러블리 한 우정 ',
 ' 블랙핑크 킬 디스 러브 뮤비 18억뷰 ',
 ' 왜 김연아 먼저 블랙핑크 지수 해외 팬들 뿔났다 ',
 ' 블랙핑크 지수 가시지 않은 꽃 향기 여운 ',
 '블랙핑크 이을 동생 그룹 베이비몬스터 데뷔 YG에 꽂힌 외국인',
 ' 블랙핑크 Kill This Love MV 18억뷰 돌파 ',
 ' 왜 김연아보다 뒤에 블랙핑크 지수 팬들 뿔났다 ',
 '블랙핑크 콘서트 간 연예인 블랙리스트 올랐다',
 ' 사진 블랙핑크 아닌 블랙스완 ',
 ' 블랙핑크 로제 잘 다녀왔어요 포토 ',
 ' 블랙핑크 제니 내추럴 뷰티 포토 ',
 ' 블랙핑크 제니 눈밑 상처 잘 아물었네 ',
 ' 블랙핑크 제니 슈퍼스타의 초근접 셀카 귀여움 한 가득 ',
 ' 블랙핑크 지수 문틈사이로 인사 포

In [ ]:
test_content = []
for content in test['content']:
    pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
    result = pat.sub(" ",content)
    result = re.sub(" +", " ", result)
    test_content.append(result)

test_content

[' 블랙핑크 더 게임 지난 18일 글로벌 출시 블랙핑크 매력 보여줄 수 있는 게임 만들기 위해 노력 블랙핑크가 또 한번 게임에 등장했다 멤버들의 모습을 담은 블랙핑크 더 게임 이 전 세계 팬들을 만나는 중이다 YG엔터테인먼트 제공그룹 블랙핑크가 또 한번 게임에 등장했다 멤버 제니 지수 로제 리사의 모습을 담은 블랙핑크 더 게임 이 전 세계 팬들을 만나는 중이다 블랙핑크 IP 기반의 새 모바일 게임 블랙핑크 더 게임 은 지난 18일 글로벌 출시됐다 이 게임에서는 멀티버스 세계 속 블랙핑크의 프로듀서가 돼 멤버들을 성장시킬 수 있다 게임 제작사 테이크원컴퍼니 측은 글로벌 아티스트 블랙핑크의 매력을 최대한 보여줄 수 있는 게임을 만들기 위해 노력했다 고 밝힌 바 있다 게임 속 멤버들은 밝은 목소리로 이용자에게 인사를 건넨다 이후 이용자는 닉네임을 입력해 자신의 프로듀서 명함을 만들게 된다 혼돈의 조각 탓에 블랙핑크가 데뷔하기 전으로 세상이 틀어졌다는 설정 속에서 사용자는 음반 보관실 음반 제작실 굿즈 개발실 등의 부서를 만들고 테디 다운 MD 매니저 건 등의 캐릭터들과 소통하게 된다 스케줄을 진행하는데 필요한 게임 속 포토카드는 팬들을 위한 볼거리 중 하나다 블랙핑크 월드라는 곳도 마련돼 있다 이곳에서는 이모티콘 이모션을 통해 다른 이들과 소통할 수 있다 해외 팬들도 눈에 띈다 월드의 스타일숍을 통해서는 아바타 놀이를 할 때처럼 멤버들을 예쁜 헤어스타일과 옷 화관 가방 등으로 꾸며주는 일이 가능하다 블랙핑크를 향한 뜨거운 관심을 증명하듯 블랙핑크 더 게임 은 구글 플레이 애플 앱스토어에서 금주의 추천 게임 으로 선정됐다 블랙핑크의 적극적인 게임 활용 블랙핑크는 배틀그라운드 모바일 인게임 콘서트 스페셜 트랙인 레디 포 러브 의 3D 아바타 뮤직비디오를 공개한 바 있다 YG엔터테인먼트 제공블랙핑크는 이전에도 게임으로 대중을 만났다 메타버스 플랫폼 제페토에는 이들과 관련된 맵이 구현돼 있다 블핑하우스 맵을 찾은 사용자들은 이 공간의 블랙핑크 사진 영상을 구경하거나 엔터

In [ ]:
test['title'] = test_title
test['content'] = test_content
test.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오후 9:17,블랙핑크와 게임의 만남,https://n.news.naver.com/mnews/article/469/000...,블랙핑크 더 게임 지난 18일 글로벌 출시 블랙핑크 매력 보여줄 수 있는 게임 만...,NaN,NaN
1,2023-05-31 20:00:01,쉬운 우리말 쓰기 블랙핑크가 맡은 헤드라이너 가 뭔가요,https://n.news.naver.com/mnews/article/001/001...,쉬운 우리말 쓰기 블랙핑크가 맡은 헤드라이너 가 뭔가요 편집자 주 우리 말과 글은...,NaN,NaN
2,2023-05-30 08:29:03,유안타 와이지엔터 블랙핑크 성장세 지속,https://n.news.naver.com/mnews/article/003/001...,자료제공 유안타증권 재판매 및 DB 금지 서울 뉴시스 이지영 기자 유안타증권은 3...,NaN,NaN
3,2023.05.30. 오전 9:05,블랙핑크 여동생 베이비 몬스터 심상치 않은 인기 글로벌팬 공감대,https://n.news.naver.com/mnews/article/112/000...,헤럴드POP 김지혜 기자 YG 신인 걸그룹 베이비몬스터 BABYMONSTER 가 ...,NaN,NaN
4,2023.05.31. 오후 3:11,블랙핑크 뉴진스 전에 K맘 행복배틀 이엘 진서연의 스릴러 종합,https://n.news.naver.com/mnews/article/311/000...,엑스포츠뉴스 김현정 기자 행복배틀 이 치열한 심리전과 예측불가 전개를 보여줄 것으...,NaN,NaN


In [ ]:
# 제목, 본문내용 합치기
test_text = test['title'] + test['content']
test_text

0       블랙핑크와 게임의 만남  블랙핑크 더 게임 지난 18일 글로벌 출시 블랙핑크 매력...
1       쉬운 우리말 쓰기 블랙핑크가 맡은 헤드라이너 가 뭔가요  쉬운 우리말 쓰기 블랙핑...
2      유안타 와이지엔터 블랙핑크 성장세 지속  자료제공 유안타증권 재판매 및 DB 금지 ...
3       블랙핑크 여동생 베이비 몬스터 심상치 않은 인기 글로벌팬 공감대  헤럴드POP 김...
4       블랙핑크 뉴진스 전에 K맘 행복배틀 이엘 진서연의 스릴러 종합  엑스포츠뉴스 김현...
                             ...                        
175    스타벅스 신메뉴 블랙핑크 콜라보 제품 후기 디저트 먹고 꿀팁스벅앱을 들어가니 뜬 소...
176    블랙핑크 제니 열애설 상기 시그널 방탄소년단 BTS 뷔와 파리 데이트룩 자발적 공개...
177    연예인향수 궁금하지 미스디올 블랙핑크 지수 강민경 엄정화 손석구 향수안녕하세요 다양...
178    스벅신상 블랙핑크 딸기 크림 초코번 맛있네제가 좋아하는 두 조합 블랙핑크 스타벅스 ...
179    베트남 하노이 블랙핑크 환영2층버스 팬들로 가득베트남 하노이 블랙핑크 환영2층버스 ...
Length: 1927, dtype: object